In [29]:
import cv2
from ultralytics import YOLO

import psutil
import os

import threading
import time


In [30]:
classe_choisie = "2x2_Bleue"

In [31]:
#chargement model 
model = YOLO("/home/dim/clone_repo/BrickSearch/outputs/output_models/y12m_multi_param_test2/weights/best.engine", task="detect")

In [32]:
source = '/home/dim/clone_repo/BrickSearch/data/raw/videos/lego_video_test.mp4'

path_video_output = '/home/dim/clone_repo/BrickSearch/outputs/video_annotees'

In [33]:
WINDOW_NAME = 'Lego Tracking'
WINDOW_WIDTH = 1000   # Nouvelle largeur souhaitée
WINDOW_HEIGHT = 800  # Nouvelle hauteur souhaitée
WINDOW_POS_X = 1900   # Position horizontale sur l'écran
WINDOW_POS_Y = 100   # Position verticale sur l'écran

In [34]:
def afficher_infos_systeme():
    # Infos RAM
    mem_info = psutil.virtual_memory()
    print("")
    print(f"Mémoire totale : {mem_info.total / (1024**2):.2f} Mo")
    print(f"Mémoire utilisée : {mem_info.used / (1024**2):.2f} Mo")
    print(f"Mémoire libre  : {mem_info.free / (1024**2):.2f} Mo")
    print("")

    # Infos CPU
    cpu_usage = psutil.cpu_percent(interval=1)
    print(f"Utilisation CPU : {cpu_usage}%")
    print("")

    # Infos GPU via GPUtil
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        if not gpus:
            print("Aucun GPU détecté.")
            print("")
        else:
            for gpu in gpus:
                print(f"GPU : {gpu.name}")
                print(f" - Utilisation : {gpu.load*100:.2f}%")
                print(f" - Mémoire utilisée : {gpu.memoryUsed} Mo / {gpu.memoryTotal} Mo")
                print("")
                
    except ImportError:
        print("GPUtil n'est pas installé. Vous pouvez l'installer via '!pip install gputil'.")


In [35]:
# Open the video file
video_path = source
cap = cv2.VideoCapture(video_path)

afficher_infos_systeme()
test = True

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    
    if test:
        afficher_infos_systeme()
        test = False

    if success:
        # Run  tracking on the frame, persisting tracks between frames
        results = model(
                                    source=frame,
                                    conf=0.5, 
                                    iou=0.5, 
                                    classes=18, 
                                )

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Création et configuration de la fenêtre
        cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(WINDOW_NAME, WINDOW_WIDTH, WINDOW_HEIGHT)
        cv2.moveWindow(WINDOW_NAME, WINDOW_POS_X, WINDOW_POS_Y)

        # Display the annotated frame
        cv2.imshow(WINDOW_NAME, annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

afficher_infos_systeme()


Mémoire totale : 32012.65 Mo
Mémoire utilisée : 10826.96 Mo
Mémoire libre  : 17941.58 Mo

Utilisation CPU : 0.3%

GPU : NVIDIA GeForce RTX 3060
 - Utilisation : 8.00%
 - Mémoire utilisée : 2255.0 Mo / 12288.0 Mo


Mémoire totale : 32012.65 Mo
Mémoire utilisée : 10828.61 Mo
Mémoire libre  : 17939.94 Mo

Utilisation CPU : 0.4%

GPU : NVIDIA GeForce RTX 3060
 - Utilisation : 10.00%
 - Mémoire utilisée : 2268.0 Mo / 12288.0 Mo

Loading /home/dim/clone_repo/BrickSearch/outputs/output_models/y12m_multi_param_test2/weights/best.engine for TensorRT inference...
[03/28/2025-10:16:49] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[03/28/2025-10:16:49] [TRT] [I] Loaded engine size: 95 MiB
[03/28/2025-10:16:49] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +93, now: C